# Voltage Glitch Examples

In [2]:
import sys
sys.path.insert(0, '..')
import securec
from securec import util
scope, target = util.init()

See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


In [3]:
import os
if 'nt' in os.name:
    os.environ['PATH'] = r'C:\cw\cw\usr\bin;C:\cw\cw\home\portable\avrgcc\avr-gcc-10.1.0-x64-windows\bin;' + os.environ['PATH']

In [4]:
securec.util.compile_and_flash('./glitch_examples.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2307 bytes
✓


In [5]:
# scope.default_setup()
scope.glitch.clk_src = 'clkgen'
scope.glitch.trigger_src = 'ext_single'
scope.glitch.output = 'glitch_only'

scope.clock.clkgen_freq = 32E6
target.baud = 230400*32/7.37
scope.io.glitch_lp = True
scope.io.glitch_hp = True

In [6]:
import logging
logging.getLogger('ChipWhisperer Target').setLevel(logging.CRITICAL)
logging.getLogger('ChipWhisperer Scope').setLevel(logging.CRITICAL)

In [14]:
import itertools
from tqdm.notebook import tqdm
import numpy as np

def frange(start, end=None, inc=1):
    if isinstance(start, np.ndarray):
        return start
    if end is None:
        end = start
    return np.arange(start, end + inc, inc)

def glitchi(widths, offsets, ext_offsets, repeats, trials, write_params, read_params, success_criterion, print_reset=True, display_progress=True, challenge_success=100):
    iteration = list(itertools.product(
        frange(widths),
        frange(offsets),
        frange(ext_offsets),
        frange(repeats),
        frange(trials),
    ))
    if display_progress:
        iteration = tqdm(iteration)

    scope.adc.timeout = 0.1
    successes = []
    util.reset_target()
    for width, offset, ext_offset, repeat, _ in iteration:
        data = (width, offset, ext_offset, repeat)

        scope.glitch.repeat = repeat
        scope.glitch.offset = offset
        scope.glitch.width = width
        scope.glitch.ext_offset = ext_offset

        if scope.adc.state:
            if print_reset:
                print('reset ', *data)
            util.reset_target()

        scope.arm()
        target.simpleserial_write(*write_params)

        scope.io.glitch_hp = False
        scope.io.glitch_hp = True
        scope.io.glitch_lp = False
        scope.io.glitch_lp = True
        ret = scope.capture()
        val = target.simpleserial_read_witherrors(*read_params, glitch_timeout=0.1, timeout=0.1)

        if ret:
            if print_reset:
                print('reset ', *data)
            util.reset_target()
        else:
            if val['valid'] is False:
                if print_reset:
                    print('reset ', *data)
            else:
                if val['payload'] and success_criterion(val['payload']):
                    if challenge_success != None:
                        challenged = glitchi(
                            *data,
                            trials=frange(1, challenge_success),
                            write_params=write_params,
                            read_params=read_params,
                            success_criterion=lambda payload: payload == val['payload'],
                            print_reset=False,
                            display_progress=False,
                            challenge_success=None,
                        )
                        print('success', val['payload'], *data, '->', len(challenged) / challenge_success * 100, '%')
                        successes.append((*data, len(challenged) / challenge_success))
                    else:
                        successes.append(data)
                else:
                    pass
    return successes


In [15]:
util.reset_target()
target.simpleserial_write(0x01, b'')
target.simpleserial_read_witherrors(0x01, 2)

{'valid': True,
 'payload': CWbytearray(b'0f af'),
 'full_response': CWbytearray(b'00 01 02 0f af cc 00'),
 'rv': bytearray(b'\x00')}

## Find sweet spots

All devices behave a bit different. In order to find some sweet spots a easy glitch example can be used with broad search parameters.

In [31]:
glitchi(
    widths=frange(-49, 49, 0.5),
    offsets=frange(0.5, 49, 0.5),
    ext_offsets=frange(0, 10),
    repeats=frange(20, 20),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] != 0xf and payload[1] == 0xaf,
    print_reset=False,
    challenge_success=10,
)

  0%|          | 0/212366 [00:00<?, ?it/s]

success CWbytearray(b'04 af') -43.5 37.5 0 20 -> 10.0 %
success CWbytearray(b'04 af') -43.5 38.0 5 20 -> 0.0 %
success CWbytearray(b'04 af') -43.0 8.0 1 20 -> 10.0 %
success CWbytearray(b'04 af') -43.0 21.5 5 20 -> 0.0 %
success CWbytearray(b'04 af') -43.0 27.5 1 20 -> 0.0 %
success CWbytearray(b'04 af') -43.0 30.5 3 20 -> 0.0 %
success CWbytearray(b'04 af') -43.0 31.0 0 20 -> 10.0 %
success CWbytearray(b'04 af') -43.0 31.0 3 20 -> 10.0 %
success CWbytearray(b'04 af') -43.0 32.0 0 20 -> 50.0 %
success CWbytearray(b'04 af') -43.0 32.0 5 20 -> 10.0 %
success CWbytearray(b'04 af') -43.0 32.5 3 20 -> 10.0 %
success CWbytearray(b'04 af') -43.0 34.0 0 20 -> 70.0 %
success CWbytearray(b'04 af') -43.0 36.0 0 20 -> 40.0 %
success CWbytearray(b'05 af') -43.0 36.5 3 20 -> 0.0 %
success CWbytearray(b'04 af') -42.5 1.0 4 20 -> 10.0 %
success CWbytearray(b'04 af') -42.5 1.0 5 20 -> 10.0 %
success CWbytearray(b'04 af') -42.5 2.0 1 20 -> 20.0 %
success CWbytearray(b'04 af') -42.5 2.0 6 20 -> 20.0 %
su

success CWbytearray(b'04 af') 45.0 4.5 4 20 -> 0.0 %
success CWbytearray(b'04 af') 45.0 5.0 5 20 -> 0.0 %
success CWbytearray(b'04 af') 45.5 0.5 4 20 -> 10.0 %
success CWbytearray(b'04 af') 45.5 1.5 6 20 -> 0.0 %
success CWbytearray(b'04 af') 45.5 2.0 1 20 -> 10.0 %
success CWbytearray(b'04 af') 45.5 2.5 6 20 -> 0.0 %
success CWbytearray(b'04 af') 45.5 3.0 1 20 -> 20.0 %
success CWbytearray(b'04 af') 45.5 4.5 1 20 -> 40.0 %
success CWbytearray(b'04 af') 45.5 5.0 1 20 -> 50.0 %
success CWbytearray(b'04 af') 45.5 5.0 7 20 -> 10.0 %
success CWbytearray(b'04 af') 45.5 5.5 1 20 -> 70.0 %
success CWbytearray(b'04 af') 45.5 6.0 1 20 -> 80.0 %
success CWbytearray(b'04 af') 45.5 6.5 1 20 -> 80.0 %
success CWbytearray(b'04 af') 45.5 6.5 3 20 -> 10.0 %
success CWbytearray(b'04 af') 45.5 7.0 2 20 -> 20.0 %
success CWbytearray(b'04 af') 45.5 7.5 1 20 -> 60.0 %
success CWbytearray(b'04 af') 45.5 8.5 1 20 -> 70.0 %
success CWbytearray(b'04 af') 45.5 8.5 3 20 -> 10.0 %
success CWbytearray(b'04 af') 45

[(-43.5, 37.5, 0, 20, 0.1),
 (-43.5, 38.0, 5, 20, 0.0),
 (-43.0, 8.0, 1, 20, 0.1),
 (-43.0, 21.5, 5, 20, 0.0),
 (-43.0, 27.5, 1, 20, 0.0),
 (-43.0, 30.5, 3, 20, 0.0),
 (-43.0, 31.0, 0, 20, 0.1),
 (-43.0, 31.0, 3, 20, 0.1),
 (-43.0, 32.0, 0, 20, 0.5),
 (-43.0, 32.0, 5, 20, 0.1),
 (-43.0, 32.5, 3, 20, 0.1),
 (-43.0, 34.0, 0, 20, 0.7),
 (-43.0, 36.0, 0, 20, 0.4),
 (-43.0, 36.5, 3, 20, 0.0),
 (-42.5, 1.0, 4, 20, 0.1),
 (-42.5, 1.0, 5, 20, 0.1),
 (-42.5, 2.0, 1, 20, 0.2),
 (-42.5, 2.0, 6, 20, 0.2),
 (-42.5, 3.0, 6, 20, 0.1),
 (-42.5, 3.0, 7, 20, 0.2),
 (-42.5, 6.0, 1, 20, 0.3),
 (-42.5, 7.0, 1, 20, 0.3),
 (-42.5, 8.0, 1, 20, 0.5),
 (-42.5, 10.0, 6, 20, 0.0),
 (-42.5, 10.5, 6, 20, 0.3),
 (-42.5, 11.5, 1, 20, 0.3),
 (-42.5, 11.5, 4, 20, 0.2),
 (-42.5, 12.0, 1, 20, 0.3),
 (-42.5, 12.0, 3, 20, 0.1),
 (-42.5, 12.0, 6, 20, 0.1),
 (-42.5, 12.5, 6, 20, 0.3),
 (-42.5, 13.0, 1, 20, 0.7),
 (-42.5, 13.5, 3, 20, 0.0),
 (-42.5, 14.0, 7, 20, 0.1),
 (-42.5, 14.5, 1, 20, 0.2),
 (-42.5, 15.0, 1, 20, 0.7),
 (

## Example 1

The first example addresses gives successive answers to the following questions:
- Is it possible to skip instructions? (hopefully yes ;-))
- It it possible to skip exactly one instruction?
- It it possible to skip exactly two instructions?
- How does typical "skip-patterns" look like?

To give an answer the following code shall be tested:

```c
uint16_t cnt = 0xaf00;
trigger_high();
asm("nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    "ori %0, 0x01 \n"
    "ori %0, 0x02 \n"
    "ori %0, 0x04 \n"
    "ori %0, 0x08 \n"
    "nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    "nop\n"
    : "+w"(cnt));
trigger_low();
simpleserial_put(0x01, 2, (uint8_t *)&cnt);
```

Obviously, if the result is different to `0xaf0f` an inserted glitch skipped some instruction(s).

### Test 1

Try to skip exactly one instruction.

In [ ]:
glitchi(
    widths=frange(3),
    offsets=frange(-9),
    ext_offsets=frange(0, 5),
    repeats=frange(1),
    trials=frange(1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] in (0x0e, 0x0d, 0x0b, 0x07),
    print_reset=False,
    challenge_success=200,
)

### Test 2

Try to skip either the first 2 or the last 2 ORs.

In [ ]:
glitchi(
    widths=frange(1, 48),
    offsets=frange(-40, -1),
    ext_offsets=frange(0, 0),
    repeats=frange(5, 10),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] in (0x03, 0x0c),
    print_reset=False,
    challenge_success=200,
)

### Test 3

Try to skip 3 consecutive instructions.

In [ ]:
glitchi(
    widths=frange(1, 10, 1),
    offsets=frange(-30, -1, 1),
    ext_offsets=frange(0, 10),
    repeats=frange(10, 20),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] in (0x01, 0x08),
    # success_criterion=lambda payload: payload[0] != 0xf,
    print_reset=False,
    challenge_success=200,
)

In [ ]:
glitchi(
    widths=frange(3, 4),
    offsets=frange(-2, -1),
    ext_offsets=frange(2, 3),
    repeats=frange(16, 17),
    trials=frange(1, 15),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] == 0x01,
    print_reset=False,
    challenge_success=200,
)


In [ ]:
glitchi(
    widths=frange(3, 3),
    offsets=frange(-6),
    ext_offsets=frange(1, 3),
    repeats=frange(10, 12),
    trials=frange(1, 1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] == 0x08,
    # success_criterion=lambda payload: payload[0] != 0xf,
    print_reset=False,
    challenge_success=200,
)


### Test 4

Try to skip 4 consecutive instructions.

In [ ]:
glitchi(
    widths=frange(3),
    offsets=frange(-6),
    ext_offsets=frange(0),
    repeats=frange(10),
    trials=frange(1),
    write_params=(0x01, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] == 0 and payload[1] == 0xaf,
    print_reset=False,
    challenge_success=200,
)

## Example 2

This example tries to execute a single instruction _twice_. The following code is executed:

```asm
uint8_t example2(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t *in) {
 270:	f9 01       	movw	r30, r18
  uint16_t cnt = 0xaf00;
  *(volatile uint16_t *)in = cnt;
 272:	80 e0       	ldi	r24, 0x00	; 0
 274:	9f ea       	ldi	r25, 0xAF	; 175
 276:	80 83       	st	Z, r24
 278:	91 83       	std	Z+1, r25	; 0x01
  trigger_high();
 27a:	a0 e0       	ldi	r26, 0x00	; 0
 27c:	b6 e0       	ldi	r27, 0x06	; 6
 27e:	21 e0       	ldi	r18, 0x01	; 1
 280:	15 96       	adiw	r26, 0x05	; 5
 282:	2c 93       	st	X, r18
 284:	15 97       	sbiw	r26, 0x05	; 5
  asm("nop\n"
 286:	00 00       	nop
 288:	00 00       	nop
 28a:	00 00       	nop
 28c:	00 00       	nop
 28e:	00 00       	nop
 290:	03 96       	adiw	r24, 0x03	; 3
 292:	00 00       	nop
 294:	00 00       	nop
 296:	00 00       	nop
 298:	00 00       	nop
 29a:	00 00       	nop
 29c:	00 00       	nop
 29e:	00 00       	nop
 2a0:	00 00       	nop
 2a2:	00 00       	nop
 2a4:	00 00       	nop
 2a6:	00 00       	nop
 2a8:	00 00       	nop
 2aa:	00 00       	nop
 2ac:	00 00       	nop
 2ae:	00 00       	nop
      "nop\n"
      "nop\n"
      "nop\n"
      "nop\n"
      : "+w"(cnt));
  trigger_low();
 2b0:	16 96       	adiw	r26, 0x06	; 6
 2b2:	2c 93       	st	X, r18
  *(uint16_t *)in = cnt;
 2b4:	80 83       	st	Z, r24
 2b6:	91 83       	std	Z+1, r25	; 0x01
  simpleserial_put(0x01, 2, in);
 2b8:	af 01       	movw	r20, r30
 2ba:	62 e0       	ldi	r22, 0x02	; 2
 2bc:	81 e0       	ldi	r24, 0x01	; 1
 2be:	0e 94 1a 02 	call	0x434	; 0x434 <simpleserial_put>
  return 0;
}
 2c2:	80 e0       	ldi	r24, 0x00	; 0
 2c4:	08 95       	ret
```

We target the addition `adiw r24, 0x03`. If it is executed once the expected result sent back is `03 af`. A second execution of the same instruction results in `06 af`.

In [ ]:
glitchi(
    widths=frange(2, 40, 2),
    offsets=frange(-45, 2, 2),
    ext_offsets=frange(0, 1),
    repeats=frange(1, 30),
    trials=frange(1),
    write_params=(0x02, b''),
    read_params=(0x01, 2),
    success_criterion=lambda payload: payload[0] > 3 and payload[1] == 0xaf,
    print_reset=False,
    challenge_success=200,
)

### Result
It was shown that the following parameters allow to execute a single instruction twice:

- `success CWbytearray(b'06 af') 28 -31 0 8 -> 13.5 %`
- `success CWbytearray(b'06 af') 30 -33 0 13 -> 11.5 %`

Unclear results:
- `success CWbytearray(b'05 af') 8 1 0 12 -> 55.50000000000001 %`
- `success CWbytearray(b'05 af') 8 1 1 10 -> 44.5 %`
- `success CWbytearray(b'3f af') 2 -5 0 6 -> 11.0 %`
- `success CWbytearray(b'3f af') 2 -5 0 9 -> 12.0 %`
- `success CWbytearray(b'af af') 10 -7 0 18 -> 14.499999999999998 %`
- `success CWbytearray(b'af af') 10 -7 1 15 -> 11.5 %`